In [1]:
import pandas as pd
import glob
import os
from scipy.stats import zscore
from utils import find_nulls, add_ratio_features, add_z_score

In [2]:
PATH_SAMPLE = '../dataset/real-time/fill_fold/3.csv'
CSV_FOLDER = '../dataset/real-time/fill_fold'
CSV_PREPROCESSED_FOLDER = '../dataset/real-time/fill_fold_preprocessed'

Z_SCORE_COLS =  ['trace_involved_amt','contract_block_involved','contract_tx_count','contract_main_active_days',
                'sender_block_involved','sender_tx_count','sender_main_active_days','contract_interact',
                'sender_tx_count_call_contract','sender_days_call_contract','trace_amt','distinct_sender_in_contract',
                'contract_lifetime_days','contract_lifetime_block','distinct_contract_sender_called',
                'sender_lifetime_days','sender_lifetime_block','contract_involved_amt','max_breadth','depth',
                'distinct_was_called_in_sample','distinct_sender_call_in_sample','gas','gas_price',
                'receipt_cumulative_gas_used','receipt_gas_used','value','nonce']

SUS_FILE = '../dataset/sus_tx.csv'  # Replace with the actual file path

In [3]:
def impute_depth_and_max_breadth(path):
    df = pd.read_csv(path)
    df['max_breadth'] = df['max_breadth']+1
    df['max_breadth'] = df['max_breadth'].fillna(0)
    df['depth'] = df['depth'].fillna(0)
    return df

def add_sus_col(df,attack_transactions):

    def set_value_based_on_condition(row):
        if (row['transaction_hash'] in attack_transactions):
            return 1
        else:
            return 0
        
    df['is_sus'] = df.apply(set_value_based_on_condition, axis=1)

    sus_data = df[df['is_sus'] == 1]
    not_sus_data = df[df['is_sus'] == 0]

    print(len(sus_data))
    print(len(not_sus_data))

    return df

In [4]:
sample_df = pd.read_csv(PATH_SAMPLE)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   transaction_hash                 525 non-null    object 
 1   trace_involved_amt               525 non-null    int64  
 2   from_address                     525 non-null    object 
 3   to_address                       525 non-null    object 
 4   block_timestamp                  525 non-null    object 
 5   gas                              525 non-null    int64  
 6   gas_price                        525 non-null    int64  
 7   receipt_cumulative_gas_used      525 non-null    int64  
 8   receipt_gas_used                 525 non-null    int64  
 9   value                            525 non-null    object 
 10  nonce                            525 non-null    int64  
 11  transaction_type                 0 non-null      float64
 12  contract_block_involve

In [5]:
for file_path in glob.glob(os.path.join(CSV_FOLDER, "*.csv")):
    df = pd.read_csv(file_path)
    find_nulls(df)

transaction_type    525
dtype: int64


In [6]:
for file_path in glob.glob(os.path.join(CSV_FOLDER, "*.csv")):
    filename = os.path.basename(file_path)
    preprocessed_file_path = CSV_PREPROCESSED_FOLDER+'/'+filename
    imputed_df = impute_depth_and_max_breadth(file_path)

    imputed_df['value'] = imputed_df['value'].astype('float32')

    z_score_df = add_z_score(imputed_df,Z_SCORE_COLS)

    ratio_df = add_ratio_features(z_score_df)

    sus_df = pd.read_csv(SUS_FILE)
    sus_transactions = sus_df['sus_tx']
    sus_transactions = set(sus_transactions)
    added_sus_df = add_sus_col(ratio_df,sus_transactions)

    print('Preprocessed file:',filename)
    find_nulls(z_score_df)

    added_sus_df.to_csv(preprocessed_file_path,index=False)
    

0
525
Preprocessed file: 3.csv
transaction_type    525
dtype: int64


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
